# Extração de grafos de conhecimento de bulas de medicamentos de arquivos PDF extraídos do bulário eletrônico da ANVISA


**Materiais úteis**
- [Preparing PDFs for RAGs: um comparativo entre bibliotecas](https://towardsdatascience.com/preparing-pdfs-for-rags-b1579fc697f1/)
- [Build a knowledge graph from documents using Docling](https://alain-airom.medium.com/build-a-knowledge-graph-from-documents-using-docling-8bc05e1389f7)
- [Comprehensive Guide to Chunking in LLM and RAG Systems](https://kshitijkutumbe.medium.com/comprehensive-guide-to-chunking-in-llm-and-rag-systems-c579a11ce6e2)
- [Building Knowledge Graphs with LLM Graph Transformer](https://medium.com/data-science/building-knowledge-graphs-with-llm-graph-transformer-a91045c49b59)
- [How to construct knowledge graphs](https://python.langchain.com/docs/how_to/graph_constructing/)
- [Geração com Recuperação Aumentada (RAG) em
Grafos de Conhecimento](https://books-sol.sbc.org.br/index.php/sbc/catalog/download/153/658/1179?inline=1)

In [ ]:
# Rodar somente quando estiver fazendo a preparaçaõ do ambiente de desenvolvimento para fazer o download da biblioteca
# !python3 -m spacy download pt_core_news_md

In [ ]:
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from src.data_processing.pdf_and_markdown_pipeline import PdfAndMarkdownPipeline
from src.data_processing.chunks import HEADERS_TO_SPLIT_ON

from src.connetion.chat_model import LLMModel
from src.kg.kg_from_text import KGFromText
from src.connetion.embeddings import EmbeddingsModel


FILE_NAME = "bula_1748876242121.pdf"
PDF_FILE_PATH = f"{RAW_DATA_DIR}/{FILE_NAME}"
MD_FILE_PATH = f"{PROCESSED_DATA_DIR}/{FILE_NAME}.md"


chunk_size = 400
chunk_overlap = 100
headers_to_split_on = HEADERS_TO_SPLIT_ON

provider = "openai"
embedding_provider = "hf"
temperature = 0.7
max_tokens = None

figsize = (10, 8)
show_node_properties = False

A classe PdfAndMarkdownPipeline possui as seguintes funcionalidades:
- Converte um arquivo pdf para markdown
- Carrega o conteúdo de um arquivo markdown e gera os chunks do arquivo
- Os chunks são gerados em duas etapas: a primeira divisão se dá pelo título das seções e depois divide o conteúdo das seções de acordo com o tamanho dos chunks configurados

In [ ]:
pdf_md_pipeline = PdfAndMarkdownPipeline()

In [ ]:
# Executar somente quando for converter um pdf para markdown
# pdf_md_pipeline.pdf_to_markdown(
#     source_file_path=PDF_FILE_PATH,
#     dest_dir_path=PROCESSED_DATA_DIR
# )

In [ ]:
# Executar sempre que for ler o conteúdo do markdown
pdf_md_pipeline.load_markdown(
    source_file_path=MD_FILE_PATH
)
pdf_md_pipeline.md_documents

In [ ]:
pdf_md_pipeline.get_chunks(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    headers_to_split_on=headers_to_split_on,
)
pdf_md_pipeline.chunk_documents

A classe LLMModel carrega o modelo LLM de acordo com o provedor de modelo configurado que pode ser:
- local
- openai
- google
- groq
- hf (para huggingface)

In [ ]:
llm = LLMModel(
    provider=provider,
    temperature=temperature,
    max_tokens=max_tokens
).llm

embedding = EmbeddingsModel(
    provider=embedding_provider,
).embeddings

In [ ]:
pdf_md_pipeline.chunk_documents[5]

A classe KGFromText gera o grafo de conhecimento a partir de um modelo LLM  utilizando o LLMGraphTransformer do langchain.

In [ ]:
kg = KGFromText(
    llm=llm,
    embeddings=embedding
)

In [ ]:
chunk_documents = [pdf_md_pipeline.chunk_documents[5]]

await kg.get_kg(
    chunk_documents=chunk_documents
)

In [ ]:
kg.graph_documents

In [ ]:
kg.plot_and_export_visualization(
    file_name=FILE_NAME,
    figsize=figsize, 
    show_node_properties=show_node_properties
)

In [ ]:
kg.save()